In [13]:
import pandas as pd
import numpy as np

In [14]:
print(ball_df.columns)

Index(['Frame', 'X', 'Y'], dtype='object')


In [16]:
# Read the CSV file
ball_df = pd.read_csv('../data/raw/ball_positions.csv')

# Strip leading/trailing spaces from column names
ball_df.columns = ball_df.columns.str.strip()

# Debugging: Print the cleaned column names
print("Cleaned Columns:", ball_df.columns)

# Interpolate missing X and Y values
ball_df[['X', 'Y']] = ball_df[['X', 'Y']].interpolate(limit_direction='both')

# Drop rows where X or Y are still NaN
ball_df = ball_df.dropna(subset=['X', 'Y'])

# Convert X and Y to float32
ball_df[['X', 'Y']] = ball_df[['X', 'Y']].astype('float32')

# Check result
print("After interpolation:")
print(ball_df.head(75))

# Save cleaned version (optional)
ball_df.to_csv("../data/results/ball_positions_interpolated.csv", index=False)

Cleaned Columns: Index(['Frame', 'X', 'Y'], dtype='object')
After interpolation:
    Frame        X       Y
0       0  1195.50  857.50
1       1  1240.00  898.00
2       2  1177.50  909.00
3       3  1034.00  908.00
4       4   931.00  908.50
..    ...      ...     ...
70     70   652.50  661.50
71     71   672.75  667.75
72     72   693.00  674.00
73     73   739.00  677.00
74     74   785.00  680.00

[75 rows x 3 columns]
